In [1]:
import sys
import json
params = {'exec_id':0,'exclusions':[]}
default = {'scenario': 'test', 'training_folder':'../..', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

In [2]:
# Add path to quetzal
sys.path.insert(0,'../../../../quetzal/')
from quetzal.model import stepmodel
import os
import numba as nb
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS

In [3]:
scenario = argv['scenario']
training_folder = argv['training_folder']
# if local. add the path to the scenario scenarios/<scenario>/
if on_lambda:
    input_folder = training_folder
else:
    input_folder = f'../scenarios/{scenario}/'
parallel_folder = os.path.join(input_folder,'parallel')
if not os.path.exists(parallel_folder):
    os.makedirs(parallel_folder)

In [4]:
argv['params']

{'exec_id': 0, 'exclusions': []}

In [14]:
exec_id = argv['params'].get('exec_id',0)
exclusions = argv['params'].get('exclusions',[])
kwargs = {'SIGMA': argv['params'].get('SIGMA',4.07),
              'BETA': argv['params'].get('BETA',3),
              'POWER': argv['params'].get('POWER',2),
              'DIFF': argv['params'].get('DIFF',True)
              }

In [7]:
import geopandas as gpd
from quetzal.engine.add_network_mapmatching import duplicate_nodes
from shapely.geometry import LineString
from quetzal.model import stepmodel
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io.quenedi import split_quenedi_rlinks

In [8]:
links = gpd.read_file(os.path.join(parallel_folder, f'links_{exec_id}.geojson'))
if 'index' in links.columns:
    links.set_index('index',inplace=True)
nodes = gpd.read_file(os.path.join(parallel_folder, f'nodes_{exec_id}.geojson'),engine='pyogrio')
if 'index' in nodes.columns:
    nodes.set_index('index',inplace=True)

In [9]:
road_links = gpd.read_file(os.path.join(input_folder,'road_links.geojson'))
road_links.set_index('index',inplace=True)
road_nodes = gpd.read_file(os.path.join(input_folder,'road_nodes.geojson'))
road_nodes.set_index('index',inplace=True)

In [10]:
print('split rlinks to oneways')
road_links = split_quenedi_rlinks(road_links)


# if already mapmatched. remove road_links_list (will be redone here)
if 'road_link_list' in  links.columns:
    print('remove road_links_list')
    links = links.drop(columns = ['road_link_list'])

split rlinks to oneways


In [11]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.road_links = road_links
sm.road_nodes = road_nodes

centroid = [*LineString(sm.nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])

sm = sm.change_epsg(crs,coordinates_unit='meter')

Reprojecting model from epsg 4326 to epsg 32618: 100%|█| 10/10 [00:00<00:00, 78.


In [15]:
sm.preparation_map_matching(sequence='link_sequence',
                            by='trip_id',
                            routing=True,
                            n_neighbors_centroid=100,
                            radius_search=500,
                            on_centroid=False,
                            nearest_method='radius',
                            n_neighbors=20,
                            distance_max=3000,
                            overwrite_geom=True,
                            overwrite_nodes=True,
                            num_cores=num_cores,
                            **kwargs)

overwrite nodes: nodes shares between different trips will be duplicated and rename
0 / 26
0 / 26
0 / 26
0 / 26
5 / 26
0 / 26
5 / 26
0 / 26
0 / 26
10 / 26
10 / 26
5 / 26
5 / 26
5 / 26
0 / 26
0 / 4
5 / 26
15 / 26
10 / 26
5 / 26
15 / 26
10 / 26
10 / 26
5 / 26
4 / 4
10 / 26
20 / 26
15 / 26
20 / 26
15 / 26
10 / 26
10 / 26
15 / 26
20 / 26
25 / 26
15 / 26
20 / 26
26 / 26
25 / 2615 / 26

26 / 26
20 / 26
15 / 26
25 / 26
26 / 26
25 / 26
26 / 26
20 / 26
20 / 26
25 / 26
26 / 26
20 / 26
25 / 26
26 / 26
25 / 26
25 / 26
26 / 26
26 / 26


In [16]:
sm.links['length'] = sm.links['geometry'].apply(lambda g: g.length)
if 'speed' in sm.links.columns:
    sm.links['time'] = sm.links['length']/sm.links['speed'] * 3.6

sm.nodes = sm.nodes.to_crs(4326)
sm.links = sm.links.to_crs(4326)

sm.links = sm.links.drop(columns=['road_a','road_b','offset_b','offset_a','road_node_list'])
sm.links['road_link_list'] = sm.links['road_link_list'].fillna('[]')
sm.links['road_link_list'] = sm.links['road_link_list'].astype(str)

sm.links.to_file(os.path.join(parallel_folder, f'links_{exec_id}.geojson'), driver='GeoJSON')
sm.nodes.to_file(os.path.join(parallel_folder, f'nodes_{exec_id}.geojson'), driver='GeoJSON')